### Settings ISI project path as the absolute path, so that import functions will work for the files.

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

### Generating a list of class_names so that will be used to train and test model

In [3]:
from utils.preprocess import get_dataloaders

train_loader, test_loader, class_names = get_dataloaders()
print(class_names)  # ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']


['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


### Training model by settings the epoch value to 10, it can be increased to make it more efficient

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, class_names = get_dataloaders()

# Initialize model
model = EmotionCNN(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train
EPOCHS = 100
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")

torch.save(model.state_dict(), "../models/emotion_cnn.pth")

Epoch 1/100 | Loss: 1467.2895 | Accuracy: 35.02%
Epoch 2/100 | Loss: 1274.5023 | Accuracy: 45.71%
Epoch 3/100 | Loss: 1176.4418 | Accuracy: 49.92%
Epoch 4/100 | Loss: 1092.6699 | Accuracy: 53.93%
Epoch 5/100 | Loss: 1009.6049 | Accuracy: 57.67%
Epoch 6/100 | Loss: 929.4665 | Accuracy: 61.27%
Epoch 7/100 | Loss: 849.8264 | Accuracy: 65.11%
Epoch 8/100 | Loss: 769.9372 | Accuracy: 68.43%
Epoch 9/100 | Loss: 686.2452 | Accuracy: 71.99%
Epoch 10/100 | Loss: 609.3328 | Accuracy: 75.44%
Epoch 11/100 | Loss: 534.2344 | Accuracy: 78.51%
Epoch 12/100 | Loss: 461.4293 | Accuracy: 82.19%
Epoch 13/100 | Loss: 390.1549 | Accuracy: 84.70%
Epoch 14/100 | Loss: 331.2841 | Accuracy: 87.12%
Epoch 15/100 | Loss: 281.1230 | Accuracy: 89.20%
Epoch 16/100 | Loss: 232.0119 | Accuracy: 91.28%
Epoch 17/100 | Loss: 202.5445 | Accuracy: 92.35%
Epoch 18/100 | Loss: 171.4607 | Accuracy: 93.59%
Epoch 19/100 | Loss: 147.9708 | Accuracy: 94.71%
Epoch 20/100 | Loss: 136.3823 | Accuracy: 95.06%
Epoch 21/100 | Loss: 113

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, class_names = get_dataloaders()

# Initialize model
model = EmotionCNN(num_classes=7).to(device)

# ✅ Load pretrained weights before retraining
checkpoint_path = "../models/emotion_cnn.pth"
if torch.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print("✅ Loaded pretrained model weights.")
else:
    print("⚠️ Pretrained model not found. Training from scratch.")

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # 👈 Smaller LR for fine-tuning

# Retrain
EPOCHS = 50  # Additional epochs
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")

# Save updated model
torch.save(model.state_dict(), "../models/emotion_cnn_retrained.pth")
print("✅ Retrained model saved.")


In [4]:
import torch
import torch.nn as nn
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_, test_loader, class_names = get_dataloaders()

# Load model
model = EmotionCNN(num_classes=len(class_names)).to(device)
model.load_state_dict(torch.load("../models/emotion_cnn.pth", map_location=device))
model.eval()

# Loss
criterion = nn.CrossEntropyLoss()

# Test loop
total, correct, test_loss = 0, 0, 0
class_correct = [0] * len(class_names)
class_total = [0] * len(class_names)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Per-class accuracy
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += (predicted[i] == label).item()
            class_total[label] += 1

# Overall accuracy
print(f"✅ Test Loss: {test_loss:.4f}")
print(f"✅ Test Accuracy: {100 * correct / total:.2f}%")

# Per-class accuracy
print("\n📊 Per-class accuracy:")
for i in range(len(class_names)):
    if class_total[i] > 0:
        acc = 100 * class_correct[i] / class_total[i]
        print(f"{class_names[i]}: {acc:.2f}%")
    else:
        print(f"{class_names[i]}: No samples in test set.")

✅ Test Loss: 1669.9781
✅ Test Accuracy: 49.25%

📊 Per-class accuracy:
angry: 39.46%
disgust: 46.85%
fear: 41.21%
happy: 65.45%
neutral: 42.58%
sad: 36.09%
surprise: 65.82%


In [ ]:
import torch
import torch.nn as nn
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_, test_loader, class_names = get_dataloaders()

# Load model
model = EmotionCNN(num_classes=len(class_names)).to(device)
model.load_state_dict(torch.load("../models/emotion_cnn_retrained.pth", map_location=device))
model.eval()

# Loss
criterion = nn.CrossEntropyLoss()

# Track results
all_preds = []
all_labels = []
total, correct, test_loss = 0, 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Overall accuracy
print(f"✅ Test Loss: {test_loss:.4f}")
print(f"✅ Test Accuracy: {100 * correct / total:.2f}%")

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# Classification report
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))
